In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from comm_agents.data.data_handler import RefExpDataset
from comm_agents.models.model_single_enc import SingleEncModel
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact

In [3]:
dataset = RefExpDataset()

In [66]:
MODEL_PATH_PRE = './models/single_enc_model_pre2020-10-26.pt'
MODEL_PATH_POST = './models/single_enc_model_post2020-10-26.pt'

In [68]:
device = torch.device('cpu')
models = [SingleEncModel(observantion_size=40,
                       lat_space_size=3,
                       question_size=2,
                       enc_num_hidden_layers=10,
                       enc_hidden_size=100,
                       dec_num_hidden_layers=10,
                       dec_hidden_size=100,
                       num_decoding_agents=4,
                       device=device) for _ in range(2)]
for m, path in zip(models, [MODEL_PATH_PRE, MODEL_PATH_POST]):
    m.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
model_pre, model_post = models

RuntimeError: Error(s) in loading state_dict for SingleEncModel:
	size mismatch for selection_bias: copying a param with shape torch.Size([4, 4]) from checkpoint, the shape in current model is torch.Size([4, 3]).
	size mismatch for enc1_out.weight: copying a param with shape torch.Size([4, 100]) from checkpoint, the shape in current model is torch.Size([3, 100]).
	size mismatch for enc1_out.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([3]).
	size mismatch for a1_in.weight: copying a param with shape torch.Size([100, 6]) from checkpoint, the shape in current model is torch.Size([100, 5]).
	size mismatch for a2_in.weight: copying a param with shape torch.Size([100, 6]) from checkpoint, the shape in current model is torch.Size([100, 5]).
	size mismatch for b1_in.weight: copying a param with shape torch.Size([100, 6]) from checkpoint, the shape in current model is torch.Size([100, 5]).
	size mismatch for b2_in.weight: copying a 

In [18]:
def create_scatter_plot(answer, opt_answer, samples=1000, pretrain=True):
    
    model =  model_pre if pretrain else model_post
    answers, lat_spaces, selection_biases = model(dataset.observations[0:1000],
                                                  dataset.questions[0:1000])
    df_answers = pd.DataFrame(answers.detach().numpy(), columns=['alpha1', 'alpha2', 'phi1', 'phi2'])
    df_opt_answers = pd.DataFrame(dataset.opt_answers.detach().numpy(),
                                  columns=['alpha1_star', 'alpha2_star', 'phi1_star', 'phi2_star'])
    df_hidden_states = pd.DataFrame(dataset.hidden_states.detach().numpy(),
                                   columns=['m1', 'm2', 'q1', 'q2'])
    df_lat_space = pd.DataFrame(lat_spaces.detach().numpy(),
                                   columns=['l1', 'l2', 'l3', 'l4'])
    df_question = pd.DataFrame(dataset.questions.detach().numpy(),
                           columns=['m_ref1', 'v_ref1', 'm_ref1', 'v_ref2'])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=df_answers[answer][0:samples],
        x=df_opt_answers[opt_answer][0:samples],
        mode='markers',
        opacity=.5))
    fig.update_layout(title_text='True labels vs prediction')
    fig.update_xaxes(title_text='Optimal answer')
    fig.update_yaxes(title_text='Predicted answer')
    fig.show()
interact(create_scatter_plot, answer=['alpha1', 'alpha2', 'phi1', 'phi2'],
        opt_answer=['alpha1_star', 'alpha2_star', 'phi1_star', 'phi2_star'],
        samples=[100, 1000, 10000])

interactive(children=(Dropdown(description='answer', options=('alpha1', 'alpha2', 'phi1', 'phi2'), value='alph…

<function __main__.create_scatter_plot(answer, opt_answer, samples=1000, pretrain=True)>

In [56]:
model_post.selection_bias

Parameter containing:
tensor([[ 9.8944,  6.5240, -5.5547,  9.7942],
        [11.2317,  5.6007,  6.7192, -5.2594],
        [-5.0887, -5.8613, -6.3344, -4.9649],
        [-5.2714, -5.3342, -6.2908, -5.2888]], requires_grad=True)

In [65]:
import plotly.express as px
px.scatter(x=df_hidden_states.q1[0:1000]*df_hidden_states.q2[0:1000],
           y=df_lat_space.l1 * df_lat_space.l2)

In [25]:
df_hidden_states

,m1,m2,q1,q2
0,2.886907e-20,3.190157e-20,3.881887e-16,-5.268712e-16
1,4.716424e-20,4.255824e-20,3.297433e-16,-4.814518e-16
2,4.007451e-20,2.639950e-20,5.569538e-16,-4.749995e-16
3,1.249199e-20,4.477453e-20,3.100031e-16,-3.586409e-16
4,1.201942e-20,4.804323e-20,7.033183e-16,-4.712755e-16
...,...,...,...,...
1041817,3.934335e-20,3.914088e-20,4.882463e-16,-5.192551e-16
1041818,3.934335e-20,3.914088e-20,4.882463e-16,-5.192551e-16
1041819,3.934335e-20,3.914088e-20,4.882463e-16,-5.192551e-16
1041820,3.934335e-20,3.914088e-20,4.882463e-16,-5.192551e-16
